# Intro to JupyterLab

This is a tutorial that will guide you through how to navigate through a Jupyter notebook, as well as how to navigate around your XDC.

First, you will need to execute all of the cells to make sure that everything runs without errors. At the top of the notebook, click on the "fast forward" icon to run all of the cells.

<ins>It is not important to view what's being ran in the cells. You only need to be concerned about the output.</ins>

In [29]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# These are pre-set so that the first three steps will work properly, as they all
# depend on each other. Remaining booleans will be set in the cells.
step1Complete = step2Complete = step3Complete = False

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - umdsecXX -c "/home/umdsecXX/notebooks/resources/save.py intro"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - umdsecXX -c "/home/umdsecXX/notebooks/resources/load.py intro"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "intro"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - umdsecXX -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - umdsecXX -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - umdsecXX -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - umdsecXX -c "mrg xdc detach xdc.umdsecXX"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - umdsecXX -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - umdsecXX -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/umdsecXX/notebooks/saves/umdsecXX_intro.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                    output0_2.clear_output()
                    display(HTML("<span style='color: green;'>Your lab has been successfully loaded.</span>"))
            elif (result.returncode == 2):
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>The intro lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

### Step 1: Create a Folder

On your XDC, sign into your ```intro``` node. To do this, type ```ssh intro```. Create a folder called ```jupyterintro``` in ```/home/umdsecXX```. Make sure that you are on your node first. Otherwise, you won't be able to continue.

In [4]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_1():
    # Required to change boolean value.
    global step1Complete, step2Complete

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step1.py', shell=True)

    if (result.returncode == 1 or (result.returncode == 0 and step2Complete)):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True
    
    else:
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Check your work and try again. Did you mistype the folder name?</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 2: Make a Text File

Navigate into the ```jupyterintro``` folder and create a file called ```jupytertest.txt``` by using the [<font color="blue">touch</font>](https://www.geeksforgeeks.org/touch-command-in-linux-with-examples/) command. Try changing the permissions by using the [<font color="blue">chmod</font>](https://www.freecodecamp.org/news/file-permissions-in-linux-chmod-command-explained/) command. You can use any permissions that you'd like.

In [5]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_2():
    # Required to change boolean value.
    global step2Complete, step3Complete

    # Loading, in case the check is slow.
    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step2.py', shell=True)

    # Note: If already completed, it cannot be accidentally "undone" since it's part of the next step.
    if (result.returncode == 2 or step3Complete) or step2Complete:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True
    elif (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>The text file is created, but you forgot to change the permissions.</span>"))
            step2Complete = False
    else:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step2Complete = False

def check_step_2(b):
    step_2()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 3: Delete the Folder

Delete the entire folder. Use the [<font color="blue">rm</font>](https://www.howtogeek.com/858815/linux-rm-command/) command to delete the folder, then check your work below. 

<u>Tip</u>: You are deleting a directory, so include a tag with your command.

In [6]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_3():
    # Required to change boolean values.
    global step1Complete, step3Complete

    # Loading, in case the check is slow.
    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step3.py /home/umdsecXX/jupyterintro'], shell=True)
    
    if (result.returncode == 1 and step1Complete):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step3Complete = False
    elif step1Complete == False:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>You need to complete Step 1 before continuing.</span>"))
            step3Complete = False
    else:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

def check_step_3(b):
    step_3()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 4: Search For a File

A file was stored in your ```intro``` node called ```findme.txt```. Its location is unknown, so you need to use the [<font color="blue">find</font>](https://www.geeksforgeeks.org/find-command-in-linux-with-examples/) command to search for it. Check the top-most hierarchy of the ```intro``` node, which is ```/```.

When you have found the file, delete it.

Additionally, there is a <u>hidden</u> file located inside of your home directory. Figure out what it is by using parameters for the [<font color="blue">ls</font>](https://www.freecodecamp.org/news/linux-chmod-chown-change-file-permissions/) command, then delete that, as well. 

**NOTE FOR LATER:** Put this in ```runlab```. This does not get created yet!

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check if the folder exists
def step_4():
    # Required to change boolean values.
    global step4Complete

    # Loading, in case the check is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step4'], shell=True)
    
    if (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step4Complete = False
    elif (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step4Complete = True

def check_step_4(b):
    step_4()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(button, output4)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 5: Retrieve Files from Online

Unix environments allow you to download information from online, such as website source code, RAW files, and more. To do this, we can use a command called [<font color="blue">wget</font>](https://www.geeksforgeeks.org/wget-command-in-linux-unix/). The ```wget``` command is straightforward for our needs, as it just requires a URL. The syntax is: ```wget [url]```

In your home directory, download this RAW file by using ```wget```: ```https://raw.githubusercontent.com/jesstess/Scrabble/master/scrabble/sowpods.txt```

Make sure that the file is named ```sowpods.txt```. This will work automatically for you, but in case your test is not passing, make sure to check the file name.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check if the folder exists
def step_5():
    # Required to change boolean values.
    global step5Complete

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step5.py'], shell=True)

    # Triggers when wget has an issue when downloading the file for validity.
    if (result.returncode == 1):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Error. File cannot be downloaded to check validity.</span>"))
            step5Complete = False
    # File isn't made.
    elif (result.returncode == 2):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>The sowpods.txt file cannot be found in your home directory.</span>"))
            step5Complete = False
    # In case the two files don't match.
    elif (result.returncode == 0):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Your sowpods.txt file does not match the text file. Delete the file and try again.</span>"))
            step5Complete = False
    # Success.
    elif (result.returncode == 3):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step5Complete = True

def check_step_5(b):
    step_5()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(button, output5)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 6: Use a Regular Expression

Regular expressions (regex) are used to find/change strings in a file. They are in programming languages, but they are also available in Unix environments. The command, [<font color="blue">grep</font>](https://www.geeksforgeeks.org/grep-command-in-unixlinux/), is used for handling text data in Linux. 

Some useful cases of using grep are:
- You have several .c files stored in a directory, but you want to know which .c files use the library ```time.h```.
- You are trying to find a specific error inside of ```/var/log```.
- You want to find all occurrences of a database name in a collection of PHP files, and you want to rename them all in one command.

From the previous step, you downloaded ```sowpods.txt```. Use the ```grep``` command and find all strings that begin with the word ```CAMP``` inside of this file. <u>Case sensitivity does not matter</u>. Once you have printed all of the strings that start with ```CAMP```, create a text file called ```output.txt``` inside of ```/home/umdsecXX```. 

<u>Tips</u>: 
- Use the following command to take your ```grep``` command and write it as a text file. This is called a redirection, which takes your output from the ```grep``` command, then writes it as a new text file: ```grep ... > /home/umdsecXX/output.txt```
- ```grep``` supports regex syntax. This will be helpful when trying to find words that start with ```CAMP```.

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the folder exists
def step_6():
    # Required to change boolean values.
    global step5Complete, step6Complete, step7Complete

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
        
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step6'], shell=True)

    # If the file was moved already, then this step must have passed.
    
    if (step7Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step6Complete = True
            
    # sowpods.txt doesn't exist.
    if (result.returncode == 1 or not step5Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>sowpods.txt doesn't exist. Did you complete the previous step yet?</span>"))
            step6Complete = False
            
    # Check fails.
    elif (result.returncode == 0):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Error. This check script failed. Please consult your professor/TA.</span>"))
            step6Complete = False

    # File cannot be found.
    elif (result.returncode == 2):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Your answer cannot be found. Did you name it as output.txt?</span>"))
            step6Complete = False

    # output.txt is different from the correct answer.
    elif (result.returncode == 4):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>output.txt exists, but is incorrect.</span>"))
            step6Complete = False

    elif (result.returncode == 3):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step6Complete = True

            # For the next step, create the new folder if it doesn't exist yet.
            subprocess.run("ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro 'mkdir -p \"Important Data\"'", shell=True)
    

def check_step_6(b):
    step_6()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(button, output6)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 7: Moving a File

After completing Step 6, a new folder was created in your home directory, named ```Important Data```. Move your ```output.txt``` file into this directory by using the [<font color="blue">mv</font>](https://www.geeksforgeeks.org/mv-command-linux-examples/) command.

<u>Tips</u>:
- There is a space in your directory name. You will have to [<font color="blue">escape space characters</font>](https://linuxhandbook.com/filename-spaces-linux/) in your command.
- You can rename files use this same command! To rename a file, use ```mv oldname newname```. You are moving it to/from the same directory, but with a different name. <u>This is not required for the lab.</u>

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the folder exists
def step_7():
    # Required to change boolean value.
    global step6Complete, step7Complete

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro "/home/.checker/step7.py \'/home/umdsecXX/Important Data/output.txt\'"', shell=True)
    
    if (result.returncode == 1 and step6Complete):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step7Complete = True
    else:
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step7Complete = False

def check_step_7(b):
    step_7()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)

# Display the output.
display(button, output7)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 8: Copying a File

Now, create a copy of the file using the [<font color="blue">cp</font>](https://www.geeksforgeeks.org/cp-command-linux-examples/). Call this file ```output_copy.txt```. Make sure that it remains in the same directory as your ```output.txt``` file. You should still be in your ```Important Data``` folder. <u>By copying the two files, they must also have the same permissions.</u>

<u>Note</u>: Physically copying all the lines and creating a new text file will not work. You will need to use the copy command.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check if the folder exists
def step_8():
    # Required to change boolean values.
    global step8Complete

    # Loading, in case the check is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step8.py'], shell=True)

    # File wasn't copied yet.
    if (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>output_copy.txt is not found.</span>"))
            step8Complete = False

    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>output.txt and output_copy.txt are not the same.</span>"))
            step8Complete = False

    elif (result.returncode == 3):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Error. Please consult your professor/TA.</span>"))
            step8Complete = False
    
    elif (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step8Complete = True

def check_step_8(b):
    step_8()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)

# Display the output.
display(button, output8)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 9: Finding the Difference Between Two Files

Upon completing Step 8, a new folder was created inside of your ```Important Data/``` directory called ```lists/```. This contains two files: ```list1.txt``` and ```list2.txt```. One line is different between the two, very similar text files. It's too difficult to find what the difference is by hand, so we will use the [<font color="blue">diff</font>](https://www.geeksforgeeks.org/diff-command-linux-examples/) command to find the difference between the two files.

After finding what the difference is between the two files, you will use a similar command from Step 6 to redirect the output to a text file. Call this file ```listdiff.txt```, and keep it inside of your ```lists/``` directory.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check if the folder exists
def step_9():
    # Required to change boolean values.
    global step9Complete

    # Loading, in case the check is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step9'], shell=True)

    # No attempt made.
    if (result.returncode == 3):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>listdiff.txt was not found. Is it in the correct directory?</span>"))
            step9Complete = False
            
    # Check fails.
    elif (result.returncode == 0):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>Error. This check script failed. Please consult your professor/TA.</span>"))
            step9Complete = False

    # Exists, but not correct.
    elif (result.returncode == 2):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>Your answer is incorrect. Double check your diff command.</span>"))
            step9Complete = False

    elif (result.returncode == 1):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step9Complete = True

def check_step_9(b):
    step_9()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)

# Display the output.
display(button, output9)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 10: Creating Tarballs

Now, you will create a tarball consisting of the ```Important Data/``` directory. Tarballs (```.tar.gz```) are similar to zip (```.zip```) files, except tarballs are more supportive in Unix environments, as they will keep their permissions when zipped and will not be compressed unless you use the ```.gz``` extension (meaning compression with g-zip).

Use the [<font color="blue">tar</font>](https://www.cyberciti.biz/faq/how-to-create-tar-gz-file-in-linux-using-command-line/) command to create a tarball containing the three files from above. Name the tarball ```data.tar.gz```, and make sure it's in your home directory.

<u>Tip</u>: Upon extracting your tarball, the first file you should see is ```Important Data/```. It's not required that you extract your tarball, but if you are getting an error, this would be a good starting point to debug. If you compressed the contents inside of ```Important Data/```, rather the folder itself, then you performed the step incorrectly.

In [13]:
# Click the button below to check your work.
step10Complete = False

# Function to check if the folder exists
def step_10():
    # Required to change boolean values.
    global step10Complete

    # Loading, in case the check is slow.
    with output10:
        output10.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step10.py', shell=True, stdout=subprocess.DEVNULL)

    # File wasn't copied yet.
    if (result.returncode == 0):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>data.tar.gz was not found in your home directory.</span>"))
            step10Complete = False

    elif (result.returncode == 2):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>Your tarball was not created correctly. Review the directions and try again.</span>"))
            step10Complete = False

    elif (result.returncode == 3):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>There is an issue with how you created your tarball. Ensure that the arguments you used are -czvf for the tar command.</span>"))
            step10Complete = False

    elif (result.returncode == 1):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step10Complete = True

def check_step_10(b):
    step_10()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output10 = widgets.Output()

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(button, output10)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 11: Secure Copying a File

We want to move ```data.tar.gz``` onto our XDC. Technically, when moving files that can be copied and pasted, it would be possible to copy/paste the contents from our terminal window to a text editor. However, when we are working with very long files, this isn't a viable approach. We can't copy and paste a ```.tar.gz``` file from a terminal, so it will be easier to use the [<font color="blue">scp</font>](https://www.geeksforgeeks.org/scp-command-in-linux-with-examples/) command. ```scp``` stands for "secure copy", and we use this whenever we want to transfer files from one system to another.

This is the syntax: 

```scp [-args] [[user@]host1:]source_file_or_directory [[user@]host2:]destination```

This command may appear a little daunting, but it's incredibly important and useful to know. Other courses may require you to transfer files from your system to a server or vice versa. The syntax is essentially: Where is the file that I want to locate, and where do I want to send it to?

You will see that on the ```intro``` node, your terminal says <font color="#2eeb10"><strong>umdsecXX@intro</strong></font>, which is a recognizable host. When you are typing commands on your XDC, <strong>umdsecXX@xdc</strong> is also a recognizable host. It's good practice to look at these names on the command line so that you know what name(s) to use when using ```scp```.

Using this information, carefully type a ```scp``` command that will transfer ```/home/umdsecXX/data.tar.gz``` from your ```intro``` node to ```/home/umdsecXX``` on your XDC.

<u>Tips</u>:
- It will be easier to type ```exit``` so that you're on <strong>umdsecXX@xdc</strong>, then using the ```scp``` command. This will mean that your source is your ```intro``` node, and your destination will be ```~```, which is the home directory on your XDC.
- When you sign into the ```intro``` node, you are typing ```ssh intro```. The terminal recognizes that ```intro``` is a host, so you do not need to include ```user@``` when using this command.
- When you are typing a local directory, such as ```~```, you do not need to include the host name. The terminal will recognize that your local path will be the source/destination.

<u>The check for this step will extract your tarball and double-check that everything from Steps 5-9 were performed correctly.</u>

In [14]:
# Click the button below to check your work.
step11Complete = False

def check_tar():
    import filecmp
    
    path = os.path.expanduser("/home/umdsecXX/data.tar.gz")
    imp_data = os.path.expanduser("Important Data/")
    ret = os.path.exists(path)

    if ret:
        # Create temp folder.
        temp = "/tmp/step11/"
        subprocess.run(f"mkdir -p {temp}", shell=True)

        try:
            # Check if the tarball contains all the correct contents.
            subprocess.run(f"tar -xzf {path} -C /tmp/step11", shell=True, check=True)

            # Checking if all files are in there. A more rigourous check to ensure all content is in the files.
            files = ["output.txt", "output_copy.txt", "lists/", "lists/list1.txt", "lists/list2.txt", "lists/listdiff.txt"]
            for name in files:
                name = imp_data + name
                if not os.path.exists(temp + name):
                    subprocess.run(f"rm -rf {temp}", shell=True)
                    # A file doesn't exist.
                    return 2

                if (name == "lists/list1.txt" or name == "lists/list2.txt"):
                    size = subprocess.run(f"sed -n '$=' {temp}{name}", shell=True, capture_output=True, text=True)
                    if (not size.stdout == "1000\n"):
                        return 4

                elif (name == "lists/listdiff.txt"):
                    f = open(temp + name, "r")
                    contents = f.read()
                    if (not bool(re.search(r'^\d{3}c\d{3}\n\< .{37}\n---\n> .{37}$', contents))):
                        return 5
                    f.close()

                elif (name == "output.txt" or name == "output_copy.txt"):
                    if (not filecmp.cmp(temp + "output.txt", temp + "output_copy.txt")):
                        return 6
                    f = open(temp + "output.txt", "r")
                    lines = f.readlines()
                    for line in lines:
                        if (not bool(re.search("^CAMP", line))):
                            return 7
                    f.close()
                            
            # All files exist.
            return 1

        except subprocess.CalledProcessError:
            # Not a valid .tar.gz file type.
            return 3
    else:
        # Tarball not created.
        return 0

# Function to check if the folder exists
def step_11():
    # Required to change boolean values.
    global step11Complete

    # Loading, in case the check is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Similar check to step10.py, except done locally.
    result = check_tar()

    # Clean-up.
    subprocess.run("rm -rf /tmp/step11", shell=True)

    # File wasn't copied yet.
    if (result == 0):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>data.tar.gz is not found in your home directory.</span>"))
            step11Complete = False

    # File is missing.
    elif (result == 2):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>One or more of the files in your tarball is missing.</span>"))
            step11Complete = False

    # One of the lists is incorrect.
    elif (result == 4):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>list1.txt or list2.txt is incorrect.</span>"))
            step11Complete = False

    # listdiff.txt is incorrect.
    elif (result == 5):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>listdiff.txt is incorrect.</span>"))
            step11Complete = False

    # Files don't match.
    elif (result == 6):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Your output.txt and output_copy.txt don't match.</span>"))
            step11Complete = False

    # Error with output.txt.
    elif (result == 7):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Your output.txt file is incorrect.</span>"))
            step11Complete = False

    elif (result == 1):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step11Complete = True

            command = "ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro '[ -f ~/numbers.txt ] && [ \"$(wc -l < ~/numbers.txt)\" -eq 500 ] && exit 1 || exit 2'"
            answer = subprocess.run(command, shell=True)

            # If the numbers.txt file is not made yet, then run the next command.
            if (answer.returncode == 2):
                command = "ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro \"awk -v loop=500 -v range=1000 'BEGIN{srand(); do {numb = 1 + int(rand() * range); if (!(numb in prev)) {print numb; prev[numb] = 1; count++}} while (count<loop)}' > numbers.txt\""
                subprocess.run(command, shell=True)

def check_step_11(b):
    step_11()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(button, output11)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 12: Piping and Sorting Content

Piping is using the output of a command as input into a second command. For example, using ```ls -l | grep ".txt"``` takes the output from ```ls -l```, then runs ```grep ".txt"``` on it. This returns the lines that contain ```.txt```.

For this step, a text file was generated and called ```numbers.txt```. It's located in your home directory. Using piping, run the [<font color="blue">sort</font>](https://www.geeksforgeeks.org/sort-command-linuxunix-examples/) command to sort the text file. Put this in a new text file called ```sortednumbers.txt```. 

It is required that you use ```grep``` for this step. You will need to type the command for completing this step in the text area below. This command will be ran in the same directory as the ```numbers.txt``` file.

<u>Tips</u>: 
- Upon running the ```sort``` command, you will notice that your ```sortednumbers.txt``` file is not sorted numerically. It's sorted by leading characters. <u>You don't need to worry about sorting it numerically for this exercise.</u> If you would like to sort it numerically, you can use the ```-n``` tag.
- Before submitting your answer, you're encouraged to try experimenting with ```sort``` on your ```intro``` node first. Once you feel like you have a command that works, you may input it below and see if your test passes.
- Recall that you are not sorting ```numbers.txt``` itself. You are sorting the text file, then outputting it to ```sortednumbers.txt```.
- If you need to reset the ```numbers.txt``` file, you can simply run the test without a command in the text entry.
- Your command will be tested on a newly generated list of numbers. 

Answer: ```cat ~/numbers.txt | sort > sortednumbers.txt```

In [28]:
# Click the button below to check your work.
step12Complete = False

# Function to check if the folder exists
def step_12():
    # Required to change boolean values.
    global step12Complete

    # Loading, in case the check is slow.
    with output12:
        output12.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, regenerate the numbers.txt file.
    command = "ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro \"awk -v loop=500 -v range=1000 'BEGIN{srand(); do {numb = 1 + int(rand() * range); if (!(numb in prev)) {print numb; prev[numb] = 1; count++}} while (count<loop)}' > numbers.txt\""
    subprocess.run(command, shell=True)

    # Second, check if the input is blank. If it is, do not do anything.
    if (userInput.value == ""):
        with output12:
            output12.clear_output()
            display(HTML("<span>numbers.txt has been reset.</span>"))

    # Otherwise, send the input to the home directory.
    else:
        # Next, make sure the command contains a pipe, does not have a comment, and contains sortedlists.txt.
        pattern = r'^(?=.*\|)(?!.*#)(?=.*\bsortednumbers\.txt\b).+$'
        if re.match(pattern, userInput.value):
            # Finally, run the command if everything looks good.
            subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro "' + userInput.value + '"', shell=True)
        
            result = subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step12.py', shell=True)
        
            # File wasn't copied yet. Might not happen because it gets checked in the regular expression.
            if (result.returncode == 0):
                output12.clear_output()
                with output12:
                    display(HTML("<span style='color: red;'>sortednumbers.txt was not found in your home directory.</span>"))
                    step12Complete = False
        
            elif (result.returncode == 2):
                output12.clear_output()
                with output12:
                    display(HTML("<span style='color: red;'>Your sortednumbers.txt file is incorrect.</span>"))
                    step12Complete = False
        
            elif (result.returncode == 1):
                output12.clear_output()
                with output12:
                    display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
                    step12Complete = True
        
                subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro "echo \'echo Congrats! You finished the lab!\' > message.sh"', shell=True)

        else:
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>Check to make sure that you are using a pipe, do not have comments, and ensure that sortednumbers.txt is in your command.</span>"))
                step12Complete = False
    
    # Auto-save.
    if (not runAllSteps):
        trigger_save()

def check_step_12(b):
    step_12()

# Creating a text area.
userInput = widgets.Text(
    placeholder='Type a command here',
    description='Command:',
    layout=widgets.Layout(width='500px')
)

# Creating the button.
button = widgets.Button(description="Run Command")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(userInput, button, output12)

Text(value='', description='Command:', layout=Layout(width='500px'), placeholder='Type a command here')

Button(description='Run Command', style=ButtonStyle())

Output()

### Step 13: Executing Files

Navigate to your home directory. A new file was generated and is called ```message.sh```. You'll want to execute the file for this step. However, the permissions need to be changed. You have already learned how to change permissions in Step 2, so change the permissions so that "user" gets executable permissions. Again, use [<font color="blue">this</font>](https://www.freecodecamp.org/news/file-permissions-in-linux-chmod-command-explained/) command to make a file executable.

You may follow [<font color="blue">this guide</font>](https://www.freecodecamp.org/news/linux-chmod-chown-change-file-permissions/)  to change the permissions so that it's executable. To execute the file, use: ```./filename``` while in the same directory as the file.

POSIX permissions is the focus of the next lab. You will learn more about what these commands will do in the near future. In case you are interested on learning more about POSIX permissions, you may read [<font color="blue">this article</font>](http://learn.leighcotnoir.com/wp-content/uploads/2016/08/POSIX-file-permissions.pdf). 

In [16]:
# Click the button below to check your work.
step13Complete = False

# Function to check if the folder exists
def step_13():
    # Required to change boolean values.
    global step12Complete, step13Complete

    # Loading, in case the check is slow.
    with output13:
        output13.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/umdsecXX/.ssh/merge_key umdsecXX@intro /home/.checker/step13.py', shell=True)

    # Step 12 wasn't done yet, so it doesn't exist.
    if (not step12Complete):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You did not complete Step 12 yet.</span>"))
            step13Complete = False
    # File wasn't copied yet.
    if (result.returncode == 0):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>message.sh was not created correctly. Please consult your professor/TA.</span>"))
            step13Complete = False

    elif (result.returncode == 2):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>Your file isn't executable. Ensure that the user has executable permissions.</span>"))
            step13Complete = False

    elif (result.returncode == 1):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step13Complete = True

def check_step_13(b):
    step_13()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output13 = widgets.Output()

# Run the command on click.
button.on_click(check_step_13)

# Display the output.
display(button, output13)

Button(description='Check File', style=ButtonStyle())

Output()

## Grading

To check your overall grade, click on the button below.

*Disclaimer:* The notebook's grading system can be easily changed. However, it's important to consider that <strong>your submission will be ran on an original copy of this notebook</strong>. If you tinkered any of the input cells to pass a step, you will not get full credit.

In [17]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12, step_13]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    # global step1Complete, step2Complete, step3Complete
    for func in steps_to_check:
        func()  # Call each function in order

    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete, step13Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Auto-save at the very end.
    runAllSteps = False
    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Create a button to refresh the cell and another to calculate grade
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions
grade_button.on_click(calculate_grade)

# Output area
gradeOutput = widgets.Output()

# Display the buttons and output
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

## Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```~/notebooks/submissions```. This folder is stored on your XDC, and will not be deleted upon stopping the lab.

In [18]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "intro"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - umdsecXX -c "bash /share/stopexp ' + labname + '"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()